In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
from qick import *
from qick.asm_v2 import AveragerProgramV2

In [2]:
### IF YOU NEED TO REPROGRAM THE CLOCKS ###
#import xrfclk 
#xrfclk.set_ref_clks(lmk_freq=245.76, lmx_freq=491.52)

### IF YOU NEED TO CLEAR CACHE ###
from pynq.pl_server.global_state import clear_global_state
clear_global_state()

In [3]:
soc = QickSoc(bitfile="/home/xilinx/jupyter_notebooks/qick/qick_demos/custom/drivers/216_tProc_v8.bit")
soccfg = soc
print(soccfg)

QICK running on ZCU216, software version 0.2.297

Firmware configuration (built Thu Nov 14 10:01:25 2024):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	9 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.838 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 16384 samples (1.709 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (3.419 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 2, blk 2 is 2_230, on JHC3
	3:	axis_sg_int4_v2 - envelope memory 8192 samples (19.048 us)
		fs=6881.280 MHz, fabric=430.080 MHz, 32-bit DDS, range=1720.320 MHz
		DAC tile 1, blk 0 is 0_229, on JHC1
	4:	axis_sg_mixmux8_v1 - envelope memory 0 samples (0.000 us)
		fs=6881.280 MHz, fabric=430.080 MHz, 32-b

In [4]:
type(soc)

qick.qick.QickSoc

In [5]:
soccfg = QickConfig(soc.get_cfg())

In [6]:
type(soccfg)

qick.qick_asm.QickConfig

In [7]:
print(soccfg)

QICK running on ZCU216, software version 0.2.297

Firmware configuration (built Thu Nov 14 10:01:25 2024):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	9 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.838 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 16384 samples (1.709 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (3.419 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 2, blk 2 is 2_230, on JHC3
	3:	axis_sg_int4_v2 - envelope memory 8192 samples (19.048 us)
		fs=6881.280 MHz, fabric=430.080 MHz, 32-bit DDS, range=1720.320 MHz
		DAC tile 1, blk 0 is 0_229, on JHC1
	4:	axis_sg_mixmux8_v1 - envelope memory 0 samples (0.000 us)
		fs=6881.280 MHz, fabric=430.080 MHz, 32-b

In [9]:
# Since we're running locally on the QICK, we don't need a separate QickConfig object.
# If running remotely, you could generate a QickConfig from the QickSoc:
#     soccfg = QickConfig(soc.get_cfg())
# or save the config to file, and load it later:
with open("qick_config.json", "w") as f:
    f.write(soc.dump_cfg())
    # soccfg = QickConfig("qick_config.json")
# soccfg = soc
# print(soccfg)

In [4]:
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/qick/qick_demos/custom/drivers')

from TIDAC80508 import TIDAC80508
tidac = TIDAC80508()

In [5]:
### SET POWER FOR DACs ###
dac_2280 = soccfg.usp_rf_data_converter_0.dac_tiles[0].blocks[0]
dac_2280.SetDACVOP(40000)
dac_2281 = soccfg.usp_rf_data_converter_0.dac_tiles[0].blocks[1]
dac_2281.SetDACVOP(40000)
dac_2282 = soccfg.usp_rf_data_converter_0.dac_tiles[0].blocks[2]
dac_2282.SetDACVOP(40000)
dac_2283 = soccfg.usp_rf_data_converter_0.dac_tiles[0].blocks[3]
dac_2283.SetDACVOP(40000)
dac_2290 = soccfg.usp_rf_data_converter_0.dac_tiles[1].blocks[0]
dac_2290.SetDACVOP(40000)

dac_2292 = soccfg.usp_rf_data_converter_0.dac_tiles[1].blocks[2]
dac_2292.SetDACVOP(40000)

dac_2230 = soccfg.usp_rf_data_converter_0.dac_tiles[2].blocks[0]
dac_2230.SetDACVOP(40000)
dac_2231 = soccfg.usp_rf_data_converter_0.dac_tiles[2].blocks[1]
dac_2231.SetDACVOP(40000)
dac_2232 = soccfg.usp_rf_data_converter_0.dac_tiles[2].blocks[2]
dac_2232.SetDACVOP(40000)

In [6]:
### ENABLE MULTI TILE SYNCHRONIZATION ###
soccfg.usp_rf_data_converter_0.mts_dac_config.RefTile = 2
soccfg.usp_rf_data_converter_0.mts_dac_config.Tiles = 0b0011
soccfg.usp_rf_data_converter_0.mts_dac_config.SysRef_Enable = 1
soccfg.usp_rf_data_converter_0.mts_dac_config.Target_Latency = -1
soccfg.usp_rf_data_converter_0.mts_dac()

In [7]:
FLUX         = [0,1,2]            # DAC
PROBE_CH     = 4                  # DAC
DRIVE        = [5,6,7,8,3]        # DAC
FEEDBACK_PFB = [0,1,2,3,4,5,6,7]  # ADC
MAX_GAIN     = 32766  

In [8]:
class MuxProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_chs = cfg['ro_chs']
        gen_ch = cfg['gen_ch']
        flux_ch  = cfg['flux'][0]   
        
        self.declare_gen(ch=gen_ch, nqz=cfg['nqz'], ro_ch=ro_chs[0], mux_freqs=cfg['pulse_freqs'], mux_gains=cfg['pulse_gains'],  mux_phases=cfg['pulse_phases'], mixer_freq=cfg['mixer_freq'])
        for ch, f, ph in zip(cfg['ro_chs'], cfg['pulse_freqs'], cfg['ro_phases']):
            self.declare_readout(ch=ch, length=cfg['ro_len'], freq=f, phase=ph, gen_ch=gen_ch)
        self.add_pulse(ch=gen_ch, name="mymux", style="const", length=cfg["pulse_len_ro"], mask=[0,1,2,3,4,5,6,7])        
        
        self.declare_gen(ch=cfg['drive'][0], nqz=2, mixer_freq=4000)
        self.add_gauss(ch=cfg['drive'][0], name="gauss", sigma=cfg['pulse_len_gauss']/5, length=cfg['pulse_len_gauss'], even_length=True)
        self.add_pulse(ch=cfg['drive'][0], name="y/2", style="arb", envelope="gauss", freq=cfg['freq2'], phase=0, gain=1.0)
        
        self.declare_gen(ch=cfg['drive'][4], nqz=2, mixer_freq=4000)
        self.add_gauss(ch=cfg['drive'][4], name="gauss", sigma=cfg['pulse_len_gauss']/5, length=cfg['pulse_len_gauss'], even_length=True)
        self.add_pulse(ch=cfg['drive'][4], name="z/2", style="arb", envelope="gauss", freq=cfg['freq2'], phase=0, gain=1.0)
        
        flux_pulse_length = 16 * int(self.cfg["pulse_len_flux"] * 599.04)
        time  = np.arange(0, flux_pulse_length) * self.cfg["pulse_len_flux"] / flux_pulse_length
        flux_i = 0.5 * (1 + np.exp(0.1 * time))          
        self.declare_gen(ch=flux_ch, nqz=1)
        self.add_envelope(ch=flux_ch, name="rfpulse", idata=0.95*MAX_GAIN*flux_i, qdata=None)
        self.add_pulse(ch=flux_ch, name="x/2", style="arb", envelope="rfpulse", freq=cfg['freq1'], phase=0, gain=1.0)
        
    def _body(self, cfg):
        self.trigger(ros=cfg['ro_chs'], t=cfg['trig_time'])
        self.pulse(ch=cfg['drive'][0], name="y/2",   t=0)
        self.pulse(ch=cfg['drive'][4], name="z/2",   t=0)
        self.pulse(ch=cfg['flux'][0] , name="x/2",   t=0.02)
        self.pulse(ch=cfg['gen_ch']  , name="mymux", t=0.05)

In [9]:
config = {'drive': DRIVE,
          'flux':  FLUX,
          'gen_ch': PROBE_CH,
          'ro_chs': FEEDBACK_PFB,
          'mixer_freq': 5000,
          'nqz': 2,
          'pulse_freqs':  5000 + np.arange(8)*50,
          'pulse_gains':  [1,0,1,0,1,0,1,0],
          'pulse_phases': [0,0,0,0,0,0,0,0],
          'ro_phases':    [0,0,0,0,0,0,0,0],
          'trig_time': 0.7,
          'freq1': 0,
          'freq2': 4500,
          'pulse_len_flux' : 70/430.08,
          'pulse_len_gauss': 70/430.08,
          'pulse_len_ro': 70/430.08,
          'ro_len': 70/430.08,
         }

In [11]:
"""
prog = MuxProgram(soccfg, reps=1, final_delay=0.5, cfg=config)
iq_list = prog.acquire_decimated(soc, soft_avgs=1, progress=False)

phases = [np.angle(iq.mean(axis=0).dot([1,1j]), deg=True) for iq in iq_list]
config['pulse_phases'] = [-x for x in phases]
print("phase offsets:", config['pulse_phases'])
print("frequencies  :", config['pulse_freqs'])
"""
prog = MuxProgram(soccfg, reps=1, final_delay=0.5, cfg=config)
iq_list = prog.acquire_decimated(soc, soft_avgs=1, start_src="internal")
"""
t = prog.get_time_axis(ro_index=0)
fig, axes = plt.subplots(4, 2, figsize=(24,16))
axes[0,0].plot(t, np.abs(iq_list[0].dot([1,1j])))
axes[0,0].set_ylim(0, 400)
axes[0,0].set_title("CH1 - {} MHz".format(config['pulse_freqs'][0]))
axes[0,1].plot(t, np.abs(iq_list[1].dot([1,1j])))
axes[0,1].set_ylim(0, 400)
axes[0,1].set_title("CH2 - {} MHz".format(config['pulse_freqs'][1]))
axes[1,0].plot(t, np.abs(iq_list[2].dot([1,1j])))
axes[1,0].set_ylim(0, 400)
axes[1,0].set_title("CH3 - {} MHz".format(config['pulse_freqs'][2]))
axes[1,1].plot(t, np.abs(iq_list[3].dot([1,1j])))
axes[1,1].set_ylim(0, 400)
axes[1,1].set_title("CH4 - {} MHz".format(config['pulse_freqs'][3]))
axes[2,0].plot(t, np.abs(iq_list[4].dot([1,1j])))
axes[2,0].set_ylim(0, 400)
axes[2,0].set_title("CH5 - {} MHz".format(config['pulse_freqs'][4]))
axes[2,1].plot(t, np.abs(iq_list[5].dot([1,1j])))
axes[2,1].set_ylim(0, 400)
axes[2,1].set_title("CH6 - {} MHz".format(config['pulse_freqs'][5]))
axes[3,0].plot(t, np.abs(iq_list[6].dot([1,1j])))
axes[3,0].set_ylim(0, 400)
axes[3,0].set_title("CH7 - {} MHz".format(config['pulse_freqs'][6]))
axes[3,1].plot(t, np.abs(iq_list[7].dot([1,1j])))
axes[3,1].set_ylim(0, 400)
axes[3,1].set_title("CH8 - {} MHz".format(config['pulse_freqs'][7]))
plt.show()
"""

  0%|          | 0/1 [00:00<?, ?it/s]

'\nt = prog.get_time_axis(ro_index=0)\nfig, axes = plt.subplots(4, 2, figsize=(24,16))\naxes[0,0].plot(t, np.abs(iq_list[0].dot([1,1j])))\naxes[0,0].set_ylim(0, 400)\naxes[0,0].set_title("CH1 - {} MHz".format(config[\'pulse_freqs\'][0]))\naxes[0,1].plot(t, np.abs(iq_list[1].dot([1,1j])))\naxes[0,1].set_ylim(0, 400)\naxes[0,1].set_title("CH2 - {} MHz".format(config[\'pulse_freqs\'][1]))\naxes[1,0].plot(t, np.abs(iq_list[2].dot([1,1j])))\naxes[1,0].set_ylim(0, 400)\naxes[1,0].set_title("CH3 - {} MHz".format(config[\'pulse_freqs\'][2]))\naxes[1,1].plot(t, np.abs(iq_list[3].dot([1,1j])))\naxes[1,1].set_ylim(0, 400)\naxes[1,1].set_title("CH4 - {} MHz".format(config[\'pulse_freqs\'][3]))\naxes[2,0].plot(t, np.abs(iq_list[4].dot([1,1j])))\naxes[2,0].set_ylim(0, 400)\naxes[2,0].set_title("CH5 - {} MHz".format(config[\'pulse_freqs\'][4]))\naxes[2,1].plot(t, np.abs(iq_list[5].dot([1,1j])))\naxes[2,1].set_ylim(0, 400)\naxes[2,1].set_title("CH6 - {} MHz".format(config[\'pulse_freqs\'][5]))\naxes[3

In [12]:
print(prog)

macros:
	WriteReg(dst='s_core_w1', src=0)
	Delay(t=1.0, auto=True, gens=True, ros=True, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0xffff7b9f9f00>}, t_regs={'t': 430})
	OpenLoop(n=1, name='reps')
	Trigger(ros=[0, 1, 2, 3, 4, 5, 6, 7], pins=[], t=0.7, width=0.023251488095238096, ddr4=False, mr=False, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0xffff7b9f9e70>, 'width': <qick.asm_v2.QickParam object at 0xffff7b9f9fc0>}, t_regs={'t': 301, 'width': 10}, outdict=defaultdict(<class 'int'>, {}), trigset={6, 7, 12, 13, 14, 15, 16, 17})
	Pulse(ch=5, name='y/2', t=0, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0xffff40c8b4c0>}, t_regs={'t': 0})
	Pulse(ch=3, name='z/2', t=0, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0xffff40c8b640>}, t_regs={'t': 0})
	Pulse(ch=0, name='x/2', t=0.02, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0xffff40c8b5e0>}, t_regs={'t': 9})
	Pulse(ch=4, name='mymux', t=0.05, tag=None, t_params={'t': <qick

In [13]:
dir(prog)

['ABSOLUTE_FREQS',
 'COUNTER_ADDR',
 'FLIP_DOWNCONVERSION',
 'GAUSS_BUG',
 'REG_ALIASES',
 'USER_DURATIONS',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add_asm',
 '_add_label',
 '_apply_threshold',
 '_average_buf',
 '_body',
 '_cleanup',
 '_compile_prog',
 '_compile_waves',
 '_gen_mgrs',
 '_gen_ts',
 '_get_reg',
 '_get_wave',
 '_init_declarations',
 '_init_instructions',
 '_initialize',
 '_is_addr',
 '_make_asm',
 '_make_binprog',
 '_register_pulse',
 '_register_wave',
 '_ro_mgrs',
 '_ro_offset',
 '_ro_ts',
 'acc_buf',
 'acquire',
 'acquire_decimated',
 'add_DRAG',
 'add_cosine',
 'add_envelope'

In [15]:
print(prog.waves)

[Waveform(name='mymux_w0', freq=0, phase=0, env=0, gain=0, length=QickRawParam(par='length', start=70, spans={}, quantize=1, steps={}), conf=255), Waveform(name='y/2_w0', freq=QickRawParam(par='freq', start=1248304762, spans={}, quantize=1, steps={}), phase=QickRawParam(par='phase', start=0, spans={}, quantize=1, steps={}), env=0, gain=QickRawParam(par='gain', start=32766, spans={}, quantize=1, steps={}), length=70, conf=8), Waveform(name='z/2_w0', freq=QickRawParam(par='freq', start=1248304762, spans={}, quantize=1, steps={}), phase=QickRawParam(par='phase', start=0, spans={}, quantize=1, steps={}), env=0, gain=QickRawParam(par='gain', start=32766, spans={}, quantize=1, steps={}), length=70, conf=8), Waveform(name='x/2_w0', freq=QickRawParam(par='freq', start=0, spans={}, quantize=1, steps={}), phase=QickRawParam(par='phase', start=0, spans={}, quantize=1, steps={}), env=0, gain=QickRawParam(par='gain', start=32766, spans={}, quantize=1, steps={}), length=97, conf=8)]


In [22]:
prog.list_pulse_params('y/2')

['phase', 'gain', 'freq', 'total_length']

In [23]:
prog.envelopes

[{'next_addr': 1552,
  'envs': {'rfpulse': {'data': array([[31128,     0],
           [31128,     0],
           [31128,     0],
           ...,
           [31383,     0],
           [31383,     0],
           [31383,     0]], dtype=int16),
    'addr': 0}}},
 {'next_addr': 0, 'envs': {}},
 {'next_addr': 0, 'envs': {}},
 {'next_addr': 70,
  'envs': {'gauss': {'data': array([[ 1416,     0],
           [ 1684,     0],
           [ 1993,     0],
           [ 2346,     0],
           [ 2748,     0],
           [ 3203,     0],
           [ 3713,     0],
           [ 4284,     0],
           [ 4916,     0],
           [ 5614,     0],
           [ 6377,     0],
           [ 7208,     0],
           [ 8106,     0],
           [ 9069,     0],
           [10094,     0],
           [11178,     0],
           [12316,     0],
           [13501,     0],
           [14724,     0],
           [15977,     0],
           [17247,     0],
           [18524,     0],
           [19795,     0],
           [21

In [24]:
prog.dump_prog()

{'envelopes': [{'next_addr': 1552,
   'envs': {'rfpulse': {'data': array([[31128,     0],
            [31128,     0],
            [31128,     0],
            ...,
            [31383,     0],
            [31383,     0],
            [31383,     0]], dtype=int16),
     'addr': 0}}},
  {'next_addr': 0, 'envs': {}},
  {'next_addr': 0, 'envs': {}},
  {'next_addr': 70,
   'envs': {'gauss': {'data': array([[ 1416,     0],
            [ 1684,     0],
            [ 1993,     0],
            [ 2346,     0],
            [ 2748,     0],
            [ 3203,     0],
            [ 3713,     0],
            [ 4284,     0],
            [ 4916,     0],
            [ 5614,     0],
            [ 6377,     0],
            [ 7208,     0],
            [ 8106,     0],
            [ 9069,     0],
            [10094,     0],
            [11178,     0],
            [12316,     0],
            [13501,     0],
            [14724,     0],
            [15977,     0],
            [17247,     0],
            [18524,   